In [1]:
import torch
import os 
import sklearn 
import pandas as pd 
import numpy as np
from carla.data.catalog import CsvCatalog

/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
#TODO How to get access to test set from CARLA ? 
name='credit'
dataset = pd.read_csv(f'../data/{name}/{name}.csv')
            #TODO Better way for defining continous varaibles ?
continuous_wachter = dataset.drop(columns=['label']).columns
dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method='Identity')
dataset._df_test

,label,x1,x2,x3,x4,x6,x5,x7
6127,1.0,0.0,-12.079432,-0.266781,-0.680730,-2.670658,-2.734724,-6.367715
1583,0.0,0.0,20.577912,-0.174512,-0.601807,1.013282,-3.941916,-8.125507
7431,1.0,0.0,-7.792347,-0.003595,-3.515622,-2.430435,-6.539865,-3.330545
6437,0.0,0.0,-2.859045,-0.121479,1.911558,0.046263,-1.263214,-3.669523
5871,1.0,0.0,-6.004328,-0.128944,0.741806,-3.198453,-1.469307,-6.682411
...,...,...,...,...,...,...,...,...
4670,1.0,0.0,38.057117,-0.050271,-9.682810,5.621939,-6.371366,-0.179943
3764,1.0,0.0,13.688259,0.080665,-1.179719,2.557260,-5.688738,-1.710431
9310,1.0,0.0,-16.534884,-0.106201,-6.002221,1.062066,-10.041105,-5.081806
1101,1.0,1.0,2.422517,-0.185034,0.372868,1.084445,-1.602265,0.474350


In [3]:
model_name=['0']
accuracy=['0']
precision=['0']
recall=['0']
df=pd.DataFrame([])
for name in os.listdir('./Model/'):
    full_name=name
    model=torch.load(f'./Model/{name}')
    name=name.split('_')[1]
    name=name.replace('.pth','')
    dataset = pd.read_csv(f'../data/{name}/{name}.csv')
    continuous_wachter = dataset.drop(columns=['label']).columns
    scaler='Identity'
    if name == 'economic':
        scaler='MinMax'
    dataset = CsvCatalog(file_path=f'../data/{name}/{name}.csv',
                     continuous=continuous_wachter,
                     categorical=[],
                     immutables=[],
                     target='label',
                     scaling_method=scaler)
    dataset=dataset._df_test
    data_x, data_y=dataset.drop(columns=['label']), dataset['label']
    data_pred= model.predict(data_x)
    model_name.append(full_name)
    print(data_pred.shape)
    print(data_y.shape)
    if len(data_pred.shape)>1:
        data_pred= np.argmax(data_pred, axis=1)

    accuracy.append(sklearn.metrics.accuracy_score(data_y,data_pred))
    precision.append(sklearn.metrics.precision_score(data_y,data_pred))
    recall.append(sklearn.metrics.recall_score(data_y,data_pred))
df['model_name']=model_name
df['accuracy']=accuracy
df['precision']=precision
df['recall']=recall
df

(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/Linear_TORCH/model_linear.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(output)
/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)
(2500, 2)
(2500,)


/home/jacqueline/.local/share/virtualenvs/CARLA-7s4zdYsf/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


,model_name,accuracy,precision,recall
0,0,0,0,0
1,Forest_credit.pth,0.8004,0.831508,0.767496
2,Forest_economic.pth,0.804,0.744425,0.818868
3,Forest_sanity-3-lin.pth,0.7924,0.805,0.772182
4,Forest_sanity-3-non-add.pth,0.88,0.950617,0.52027
5,Forest_sanity-3-non-lin.pth,0.9688,0.974003,0.959044
6,Linear_credit.pth,0.7628,0.790584,0.744079
7,Linear_economic.pth,0.3652,0.347881,0.497288
8,Linear_sanity-3-lin.pth,0.996,0.996894,0.995349
9,Linear_sanity-3-non-add.pth,0.9996,1,0.998366


In [4]:
# Table to Latex 
# Get header
st=''
col=[]
name=[]
df=df.iloc[1:]
for index, row in df.iterrows():
    #print(row['model_name'].split('_'))
    if st is '':
        st += row['model_name'].split('_')[1].replace('.pth','')
    else:
        st += ' & '+row['model_name'].split('_')[1].replace('.pth','')
    col.append(row['model_name'].split('_')[0])
    name.append(row['model_name'].split('_')[1])
st+= ' \\\\ '
col= np.unique(col)
name = np.unique(name)
print(col)
print(name)
for i in range(0,len(col)):
    st += col[i] 
    for j in range(0,len(name)):

        #print()
        print(f'{col[i]}_{name[i]}')
        print(str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0]))
        #print(df[df['model_name'] ==f'{col[i]}_{name[i]}']['accuracy'])
        st += ' & ' + str(df[df['model_name'] ==f'{col[i]}_{name[j]}']['accuracy'].values[0])
    st+=' \\\\ '
print(st)

['Forest' 'Linear' 'MLP']
['credit.pth' 'economic.pth' 'sanity-3-lin.pth' 'sanity-3-non-add.pth'
 'sanity-3-non-lin.pth']
Forest_credit.pth
0.8004
Forest_credit.pth
0.804
Forest_credit.pth
0.7924
Forest_credit.pth
0.88
Forest_credit.pth
0.9688
Linear_economic.pth
0.7628
Linear_economic.pth
0.3652
Linear_economic.pth
0.996
Linear_economic.pth
0.9996
Linear_economic.pth
0.9984
MLP_sanity-3-lin.pth
0.7052
MLP_sanity-3-lin.pth
0.848
MLP_sanity-3-lin.pth
0.9952
MLP_sanity-3-lin.pth
0.986
MLP_sanity-3-lin.pth
0.9984
credit & economic & sanity-3-lin & sanity-3-non-add & sanity-3-non-lin & credit & economic & sanity-3-lin & sanity-3-non-add & sanity-3-non-lin & credit & economic & sanity-3-lin & sanity-3-non-add & sanity-3-non-lin \\ Forest & 0.8004 & 0.804 & 0.7924 & 0.88 & 0.9688 \\ Linear & 0.7628 & 0.3652 & 0.996 & 0.9996 & 0.9984 \\ MLP & 0.7052 & 0.848 & 0.9952 & 0.986 & 0.9984 \\ 
